In [8]:
import pandas as pd
import random
import requests
from collections import defaultdict
import csv
import re
import os

In [2]:
leki_forms = {
    # ból i stan zapalny
    "Ibuprom":      {"nom": "Ibuprom",      "acc": "Ibuprom",      "inst": "Ibupromem"},
    "Metafen":      {"nom": "Metafen",      "acc": "Metafen",      "inst": "Metafenem"},
    "Nurofen Forte":{"nom": "Nurofen Forte","acc": "Nurofen Forte","inst": "Nurofenem Forte"},
    "Dolgit":       {"nom": "Dolgit",       "acc": "Dolgit",       "inst": "Dolgitem"},

    # nadciśnienie
    "Bisocard":     {"nom": "Bisocard",     "acc": "Bisocard",     "inst": "Bisocardem"},
    "Bibloc":       {"nom": "Bibloc",       "acc": "Bibloc",       "inst": "Biblocem"},
    "Concor":       {"nom": "Concor",       "acc": "Concor",       "inst": "Concorem"},
    "Bisoratio":    {"nom": "Bisoratio",    "acc": "Bisoratio",    "inst": "Bisoratiem"},
    "Sobycor":      {"nom": "Sobycor",      "acc": "Sobycor",      "inst": "Sobycorem"},
    "Coronal":      {"nom": "Coronal",      "acc": "Coronal",      "inst": "Coronalem"},
    "Prestilol":    {"nom": "Prestilol",    "acc": "Prestilol",    "inst": "Prestilolem"},
    "Bisoprolol":   {"nom": "Bisoprolol",   "acc": "Bisoprolol",   "inst": "Bisoprololem"},
    "Perindopryl":  {"nom": "Perindopryl",  "acc": "Perindopryl",  "inst": "Perindoprylem"},
    "Ramipryl":     {"nom": "Ramipryl",     "acc": "Ramipryl",     "inst": "Ramiprylem"},
    "Amlodipina":   {"nom": "Amlodipina",   "acc": "Amlodipinę",   "inst": "Amlodipiną"},
    "Hydrochlorotiazyd":{"nom":"Hydrochlorotiazyd","acc":"Hydrochlorotiazyd","inst":"Hydrochlorotiazydem"},
    "Losartan":     {"nom": "Losartan",     "acc": "Losartan",     "inst": "Losartanem"},

    # cukrzyca typu drugiego
    "Avamina":      {"nom": "Avamina",      "acc": "Avaminę",      "inst": "Avaminą"},
    "Formetic":     {"nom": "Formetic",     "acc": "Formetic",     "inst": "Formeticem"},
    "Glucophage XR":{"nom": "Glucophage XR","acc": "Glucophage XR","inst": "Glucophage XR"},
    "Metformax":    {"nom": "Metformax",    "acc": "Metformax",    "inst": "Metformaxem"},
    "Xigduo":       {"nom": "Xigduo",       "acc": "Xigduo",       "inst": "Xigduo"},
    "Synjardy":     {"nom": "Synjardy",     "acc": "Synjardy",     "inst": "Synjardy"},
    "Actarosin":    {"nom": "Actarosin",    "acc": "Actarosin",    "inst": "Actarosinem"},
    "Arterios":     {"nom": "Arterios",     "acc": "Arterios",     "inst": "Arteriosem"},
    "Metforminę":   {"nom": "Metformina",   "acc": "Metforminę",   "inst": "Metforminą"},
    "Sitagliptyne": {"nom": "Sitagliptyna", "acc": "Sitagliptynę", "inst": "Sitagliptyną"},
    "Linagliptynę": {"nom": "Linagliptyna", "acc": "Linagliptynę", "inst": "Linagliptyną"},
    "Liraglutyd":   {"nom": "Liraglutyd",   "acc": "Liraglutyd",   "inst": "Liraglutydem"},
    "Dapagliflozyna":{"nom":"Dapagliflozyna","acc":"Dapagliflozynę","inst":"Dapagliflozyną"},

    # wysoki cholesterol
    "Crestor":      {"nom": "Crestor",      "acc": "Crestor",      "inst": "Crestorem"},
    "Crosuvo":      {"nom": "Crosuvo",      "acc": "Crosuvo",      "inst": "Crosuvem"},
    "Ridlip":       {"nom": "Ridlip",       "acc": "Ridlip",       "inst": "Ridlipem"},
    "Romazic":      {"nom": "Romazic",      "acc": "Romazic",      "inst": "Romazicem"},
    "Rosucard":     {"nom": "Rosucard",     "acc": "Rosucard",     "inst": "Rosucardem"},
    "Rosutrox":     {"nom": "Rosutrox",     "acc": "Rosutrox",     "inst": "Rosutroxem"},
    "Roswera":      {"nom": "Roswera",      "acc": "Roswerę",      "inst": "Roswerą"},
    "Suvardio":     {"nom": "Suvardio",     "acc": "Suvardio",     "inst": "Suvardio"},
    "Zahron":       {"nom": "Zahron",       "acc": "Zahron",       "inst": "Zahronem"},
    "Zaranta":      {"nom": "Zaranta",      "acc": "Zarantę",      "inst": "Zarantą"},
    "Rosuvastatin Medical Valley":{"nom":"Rosuvastatin Medical Valley","acc":"Rosuvastatin Medical Valley","inst":"Rosuvastatin Medical Valley"},

    # niedobór witaminy D
    "Vigantoletten": {"nom": "Vigantoletten", "acc": "Vigantoletten", "inst": "Vigantolettenem"},
    "Vigantol":      {"nom": "Vigantol",      "acc": "Vigantol",      "inst": "Vigantolem"},
    "Ibuvit D3":     {"nom": "Ibuvit D3",     "acc": "Ibuvit D3",     "inst": "Ibuvit D3"},
    "Biaron D":      {"nom": "Biaron D",      "acc": "Biaron D",      "inst": "Biaron D"},
    "ApoD3":         {"nom": "ApoD3",         "acc": "ApoD3",         "inst": "ApoD3"},
    "Devikap":       {"nom": "Devikap",       "acc": "Devikap",       "inst": "Devikap"},
    "Witamina D3 Forte": {"nom":"Witamina D3 Forte","acc":"Witaminę D3 Forte","inst":"Witaminą D3 Forte"},
    "Oriovit-D 1000": {"nom":"Oriovit-D 1000","acc":"Oriovit-D 1000","inst":"Oriovit-D 1000"},
    "Vigalex Bio":    {"nom":"Vigalex Bio","acc":"Vigalex Bio","inst":"Vigalex Bio"},
    "Vigalex Forte":  {"nom":"Vigalex Forte","acc":"Vigalex Forte","inst":"Vigalex Forte"},
    "Vigalex Max":    {"nom":"Vigalex Max","acc":"Vigalex Max","inst":"Vigalex Max"},

    # anginę
    "Ospen 1000":    {"nom":"Ospen 1000","acc":"Ospen 1000","inst":"Ospen 1000"},
    "Amotaks":       {"nom":"Amotaks","acc":"Amotaks","inst":"Amotaks"},
    "Augmentin":     {"nom":"Augmentin","acc":"Augmentin","inst":"Augmentin"},
    "Macromax":      {"nom":"Macromax","acc":"Macromax","inst":"Macromax"},
    "Sumamed":       {"nom":"Sumamed","acc":"Sumamed","inst":"Sumamed"},
    "Amoksycylinę":  {"nom":"Amoksycylina","acc":"Amoksycylinę","inst":"Amoksycyliną"},

    # zgaga
    "Omeprazol":     {"nom":"Omeprazol","acc":"Omeprazol","inst":"Omeprazolem"},
    "Pantoprazol":   {"nom":"Pantoprazol","acc":"Pantoprazol","inst":"Pantoprazolem"},
    "Laktuloza":     {"nom":"Laktuloza","acc":"Laktulozę","inst":"Laktulozą"},
     "Bisakodyl":     {"nom":"Bisakodyl","acc":"Bisakodyl","inst":"Bisakodylem"},
    "Sukralfat":     {"nom":"Sukralfat","acc":"Sukralfat","inst":"Sukralfatem"},

    # profilaktyka (suplementy)
    "Folik":         {"nom":"Folik","acc":"Folik","inst":"Folikiem"},
    "ActiFolin":     {"nom":"ActiFolin","acc":"ActiFolin","inst":"ActiFolinem"},
    "Aliness Kwas Foliowy": {"nom":"Aliness Kwas Foliowy","acc":"Aliness Kwas Foliowy","inst":"Aliness Kwasem Foliowym"},
    "Vigantol":      {"nom":"Vigantol","acc":"Vigantol","inst":"Vigantolem"},
    "D-Vitum Forte": {"nom":"D-Vitum Forte","acc":"D-Vitum Forte","inst":"D-Vitum Forte"},
    "Ibuvit D3":     {"nom":"Ibuvit D3","acc":"Ibuvit D3","inst":"Ibuvit D3"},
    "Olicaps Witamina D3": {"nom":"Olicaps Witamina D3","acc":"Olicaps Witamina D3","inst":"Olicaps Witaminą D3"},
    "OstroVit Omega 3": {"nom":"OstroVit Omega 3","acc":"OstroVit Omega 3","inst":"OstroVit Omega 3"},
    "KFD Omega 3":   {"nom":"KFD Omega 3","acc":"KFD Omega 3","inst":"KFD Omega 3"},

    # bezsenność
    "Zolpidem":      {"nom":"Zolpidem","acc":"Zolpidem","inst":"Zolpidemem"},
    "Zopiklon":      {"nom":"Zopiklon","acc":"Zopiklon","inst":"Zopiklonem"},
    "Melatoninę":    {"nom":"Melatonina","acc":"Melatoninę","inst":"Melatoniną"},
    "Trazodon":      {"nom":"Trazodon","acc":"Trazodon","inst":"Trazodonem"},
    "Mirtazapinę":   {"nom":"Mirtazapina","acc":"Mirtazapinę","inst":"Mirtazapiną"}
}

In [3]:
mapowanie = {
    "ból i stan zapalny": ['Ibuprom', 'Metafen', 'Nurofen Forte', 'Dolgit'],
    "nadciśnienie": ['Bisocard', 'Bibloc', 'Concor', 'Bisoratio', 'Sobycor', 'Coronal', 'Prestilol', 
                    'Bisoprolol',  "Perindopryl",
        "Ramipryl",
        "Amlodipina",
        "Hydrochlorotiazyd",
        "Losartan"],
    "cukrzycę typu drugiego": ['Avamina', 'Formetic', 'Glucophage XR', 'Metformax', 'Xigduo', 'Synjardy', 'Actarosin', 'Arterios',"Metforminę",
        "Sitagliptyne",
        "Linagliptynę",
        "Liraglutyd",
        "Dapagliflozyna"],
    "wysoki cholesterol": ['Crestor', 'Crosuvo', 'Ridlip', 'Romazic', 'Rosucard', 'Rosutrox', 'Roswera', 'Suvardio', 
                          'Zahron', 'Zaranta', 'Rosuvastatin Medical Valley'],
    "niedobór witaminy D": ['Vigantoletten', 'Vigantol', 'Ibuvit D3', 'Biaron D', 'ApoD3', 
                           'Devikap', 'Witamina D3 Forte', 'Oriovit-D 1000', 'Vigalex Bio'],
    "anginę": [
        "Ospen 1000",
        "Amotaks",
        "Augmentin",
        "Macromax",
        "Sumamed",
        "Amoksycylinę"
    ],
       "zgaga": [
        "Omeprazol",
        "Pantoprazol",
        "Laktuloza",
        "Bisakodyl",
        "Sukralfat"
    ],
        "profilaktyka": [
        "Folik",
        "ActiFolin",
        "Aliness Kwas Foliowy",
        "Vigantol",
        "D-Vitum Forte",
        "Ibuvit D3",
        "Olicaps Witamina D3",
        "OstroVit Omega 3",
        "KFD Omega 3"
    ],
    "bezsenność": [
        "Zolpidem",
        "Zopiklon",
        "Melatoninę",
        "Trazodon",
        "Mirtazapinę"
    ]
}


szablony_choroby = [
    "Na {choroba} przepiszę ci {lek}.",
    "Zalecam {lek}, bo dobrze działa na {choroba}.",
    "Powinnaś przyjmować {lek} na {choroba}.",
    "Jeśli masz {choroba} dobrze sprawdza się {lek}.",
    "Na recepcie zapiszę {lek} jako lek na {choroba}."
]

szablony = [
    "Możesz kupić w aptece {lek}, powinien pomóc.",
    "Przepiszę ci {lek}, bo zwykle dobrze działa.",
    "Najlepiej w twoim przypadku sprawdzi się {lek}.",
    "Na recepcie zapiszę {lek}.",
    "Powinnaś spróbować kuracji {lek}.",
]

wszystkie_szablony = szablony_choroby + szablony

In [4]:
all_sentences = []

for choroba, leki in mapowanie.items():
    for lek in leki:
        for szablon in wszystkie_szablony:

            if any(w in szablon for w in ["rzepiszę", "przyjmować", "Na recepcie", "Zalecam", "aptece"]):
                lek_forma = leki_forms.get(lek, {}).get("acc", lek)
            elif any(w in szablon for w in ["dobrze sprawdza", "sprawdzi"]):
                lek_forma = leki_forms.get(lek, {}).get("nom", lek)
            else:
                lek_forma = leki_forms.get(lek, {}).get("inst", lek)
            
            zdanie = szablon.format(choroba=choroba, lek=lek_forma)
            all_sentences.append((zdanie, lek, "male"))
            all_sentences.append((zdanie, lek, "female"))

for s in all_sentences[:10]:
    print(s)

('Na ból i stan zapalny przepiszę ci Ibuprom.', 'Ibuprom', 'male')
('Na ból i stan zapalny przepiszę ci Ibuprom.', 'Ibuprom', 'female')
('Zalecam Ibuprom, bo dobrze działa na ból i stan zapalny.', 'Ibuprom', 'male')
('Zalecam Ibuprom, bo dobrze działa na ból i stan zapalny.', 'Ibuprom', 'female')
('Powinnaś przyjmować Ibuprom na ból i stan zapalny.', 'Ibuprom', 'male')
('Powinnaś przyjmować Ibuprom na ból i stan zapalny.', 'Ibuprom', 'female')
('Jeśli masz ból i stan zapalny dobrze sprawdza się Ibuprom.', 'Ibuprom', 'male')
('Jeśli masz ból i stan zapalny dobrze sprawdza się Ibuprom.', 'Ibuprom', 'female')
('Na recepcie zapiszę Ibuprom jako lek na ból i stan zapalny.', 'Ibuprom', 'male')
('Na recepcie zapiszę Ibuprom jako lek na ból i stan zapalny.', 'Ibuprom', 'female')


In [10]:
def save_pipe_txt(all_sentences, out_path="train.txt"):
    with open(out_path, "w", encoding="utf-8", newline="") as f:
        writer = csv.writer(f, delimiter="|", quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["utt_id", "speaker", "text"])
        counts = {}
        for text, lek, speaker in all_sentences:
            counts.setdefault(lek, 0)
            counts[lek] += 1
            utt_id = f"{lek}_{counts[lek]:04d}_{speaker}"
            writer.writerow([utt_id, speaker, text])

In [11]:
sentences_by_lek = defaultdict(list)
for text, lek, speaker in all_sentences:
    sentences_by_lek[lek].append((text, lek, speaker))

train = []
dev = []
test = []

CHUNK_SIZE = 20 

for lek, records in sentences_by_lek.items():
    random.shuffle(records)
    n = len(records)
    block = records
    train.extend(block[:16])
    dev.extend(block[16:18])
    test.extend(block[18:20])

OUT_DIR = "C:/Users/Admin/Documents"

save_pipe_txt(train, os.path.join(OUT_DIR, "train.csv"))
save_pipe_txt(dev,   os.path.join(OUT_DIR, "dev.csv"))
save_pipe_txt(test,  os.path.join(OUT_DIR, "test.csv"))

In [3]:
with open("leki_nom.txt", "w", encoding="utf-8") as f:
    for lek, formy in leki_forms.items():
        f.write(formy["nom"] + "\n")